In [11]:
!open ./

In [1]:
from __future__ import division
from collections import OrderedDict
from grbTools.GENERAL.flux import Calc_Flux

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# filename = ('/Users/KimiZ/GRBs2/Sample/'
#             'Band_13_GBM+LAT__22_GBMconstrained.txt')
filename = ('/Users/KimiZ/GRBs2/Sample/'
            'Band_13_GBM__22_GBMconstrained.txt')
data = pd.read_csv(filename, sep='\t')
bursts = data.trigger.tolist()

In [3]:
data.columns

Index([u'trigger', u'name', u'number', u'z', u'LumDist', u'trigtime', u't90',
       u't90Rest', u't90start', u't90stop', u'detector', u'function',
       u'LATburst', u'flux', u'flux_err_low', u'flux_err_up', u'fluence',
       u'fluence_err_low', u'fluence_err_up', u'eiso', u'eiso_err_low',
       u'eiso_err_up', u'epeak', u'epeak_err_low', u'epeak_err_up',
       u'epeakRest', u'epeakRest_err_low', u'epeakRest_err_up', u'alpha',
       u'alpha_err_low', u'alpha_err_up', u'beta', u'beta_err_low',
       u'beta_err_up', u'norm', u'norm_err_low', u'norm_err_up', u'CSTAT',
       u'LOGLIKE', u'AIC', u'BIC', u'NDATA', u'DOF', u'Flux1', u'Flux1_err',
       u'Flux2', u'Flux2_err', u'Flux3', u'Flux3_err', u'Flue1', u'Flue1_err',
       u'Flue2', u'Flue2_err', u'Flue3', u'Flue3_err', u'Eiso1', u'Eiso1_err',
       u'Eiso2', u'Eiso2_err', u'Eiso3', u'Eiso3_err'],
      dtype='object')

In [4]:
Flux1_o = []
Flux1_r = []
Flux2_o = []
Flux2_r = []

for burst in bursts:
    
    pars = OrderedDict()
    pars['BAND'] = OrderedDict()
    pars['BAND']['alpha']= float(data[data.trigger == burst]['alpha'])
    pars['BAND']['beta']= float(data[data.trigger == burst]['beta'])
    pars['BAND']['enterm']= float(data[data.trigger == burst]['epeak'])
    pars['BAND']['norm']= float(data[data.trigger == burst]['norm'])
    pars['BAND']['entype']= 'epeak'
    
    z = float(data[data.trigger == burst]['z'])
    #duration = float(data[data.trigger == burst]['z'])
    #DL = float(data[data.trigger == burst]['LumDist'])
    
    Flux1_o.append( Calc_Flux(   model='BAND', 
                        parameters=pars, 
                        emin=1.0, 
                        emax=10000.0, 
                        redshift=None, 
                        photonflux=False
                    ))
    
    Flux1_r.append( Calc_Flux(   model='BAND', 
                        parameters=pars, 
                        emin=1.0, 
                        emax=10000.0, 
                        redshift=z, 
                        photonflux=False
                    ))
    
    Flux2_o.append( Calc_Flux(   model='BAND', 
                        parameters=pars, 
                        emin=10.0, 
                        emax=10000.0, 
                        redshift=None, 
                        photonflux=False
                    ))
    
    Flux2_r.append( Calc_Flux(   model='BAND', 
                        parameters=pars, 
                        emin=10.0, 
                        emax=10000.0, 
                        redshift=z, 
                        photonflux=False
                    ))
    
    #flux = float(data[data.trigger == burst]['flux'])
    #print('%s:  %.7E'%(burst, Flux))
    #print('%s:  %.18f'%(burst, np.diff([Flux,flux])[0]))



 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


 *** WARNING: *** 
 You are using observer-frame fluxes. Eiso requires rest-frame fluxes. Please use redshift if computing fluxes for Eiso. 


In [5]:
df = pd.DataFrame()

df['trigger'] = data['trigger']
df['z'] = data['z']
df['LumDist'] = data['LumDist']
df['t90'] = data['t90']
df['epeak'] = data['epeak']

In [6]:
df['flux_1k_10M_obsv'] = Flux1_o  
df['flux_1k_10M_rest'] = Flux1_r
df['flux_10k_10M_obsv'] = Flux2_o 
df['flux_10k_10M_rest'] = Flux2_r

In [7]:
df['flue_1k_10M_obsv'] = df['flux_1k_10M_obsv'] * df['t90']
df['flue_1k_10M_rest'] = df['flux_1k_10M_rest'] * df['t90']
df['flue_10k_10M_obsv'] = df['flux_10k_10M_obsv'] * df['t90']
df['flue_10k_10M_rest'] = df['flux_10k_10M_rest'] * df['t90']

In [8]:
df['eiso_1k_10M_obsv'] = (4.*np.pi*(df['LumDist']**2)*df['flue_1k_10M_obsv'])/(1.0+df['z'])
df['eiso_1k_10M_rest'] = (4.*np.pi*(df['LumDist']**2)*df['flue_1k_10M_rest'])/(1.0+df['z'])
df['eiso_10k_10M_obsv'] = (4.*np.pi*(df['LumDist']**2)*df['flue_10k_10M_obsv'])/(1.0+df['z'])
df['eiso_10k_10M_rest'] = (4.*np.pi*(df['LumDist']**2)*df['flue_10k_10M_rest'])/(1.0+df['z'])

In [9]:
#df.to_csv('band_results_for_sbolo_study.txt', sep='\t', index=False)
df.to_csv('band_results_for_sbolo_study_GBM.txt', sep='\t', index=False)

In [10]:
df.eiso_10k_10M_rest

0     3.456843e+54
1     4.031704e+54
2     1.131657e+53
3     3.749867e+52
4     3.417567e+54
5     1.856024e+54
6     8.788700e+52
7     2.177515e+52
8     9.150210e+53
9     5.199078e+53
10    1.780665e+54
11    5.492262e+53
12    2.007786e+53
13    3.204603e+53
14    3.572330e+53
15    2.393319e+53
16    4.182591e+52
17    2.447950e+53
18    3.915366e+52
19    1.447021e+52
20    6.357670e+52
21    4.049901e+52
22    1.276696e+53
23    7.673896e+51
24    2.573108e+53
25    8.699091e+52
26    2.471171e+52
27    3.565032e+53
28    3.561701e+52
29    1.830322e+54
30    1.940275e+53
31    1.246552e+53
32    2.827733e+52
33    3.122605e+53
34    9.646100e+52
Name: eiso_10k_10M_rest, dtype: float64

In [ ]:
!open ./

In [ ]:
plt.clf()
plt.figure(figsize=(10,7))
plt.plot( df2['epeak'], df2['flue_1k_10M_obsv'], '.')
plt.loglog()
plt.xlim(1E1, 4E3)
plt.ylim(1E-8, 1E-3)
plt.show()

In [ ]:
def calc_sbolo(yint, slope, epeak, redshift, lumdistance, fbeam=1):
    a = 10**yint
    c = slope
    k = epeak
    LD = lumdistance
    fb = fbeam
    red = redshift
    return (a * k**c * ((1.+red)**(c+1.)))/(4.*np.pi*(LD**2)*fb)